# III deo
#### Rad sa kategorijskim podacima u pandasu može značajno optimizovati performanse i uštedeti memoriju, posebno kada se radi sa kolonama koje imaju mali broj unikatnih vrednosti u poređenju sa ukupnim brojem redova. Pandas omogućava rad sa kategorijskim podacima pomoću tipa category.

In [6]:
import pandas as pd
nzl = pd.read_csv('../data/nzbiz.csv')
nzl.sample(5)

,Year,Industry_name_NZSIOC,Units,Variable_name,Variable_category,Value
343,2023,Forestry and Logging,Dollars (millions),Redundancy and severance,Financial performance,0
14272,2020,Fishing and Aquaculture,Dollars (millions),Non-operating income,Financial performance,19
50545,2013,Health Care and Social Assistance,Dollars (millions),Total expenditure,Financial performance,"12,061"
9719,2021,Mining,Dollars (millions),Current liabilities,Financial position,3235
42472,2014,"Pulp, Paper and Converted Paper Product Manufa...",Dollars (millions),Redundancy and severance,Financial performance,C


#### Kolonu sa ponavljajućim vrednostima (npr. imena gradova, tipovi proizvoda), pogodno je konvertovati u kategorijski tip:

In [11]:
nzl['Industry_name_NZSIOC'] = nzl['Industry_name_NZSIOC'].astype('category')
nzl.dtypes

Year                       int64
Industry_name_NZSIOC    category
Units                     object
Variable_name             object
Variable_category         object
Value                     object
dtype: object

#### Pregled svih kategorija:

In [12]:
nzl['Industry_name_NZSIOC'].cat.categories

Index(['Accommodation', 'Accommodation and Food Services',
       'Administrative and Support Services',
       'Adult, Community and Other Education',
       'Advertising, Market Research and Management Services',
       'Agriculture, Forestry and Fishing',
       'Agriculture, Forestry and Fishing Support Services and Hunting',
       'All industries', 'Arts and Recreation Services',
       'Arts, Recreation and Other Services',
       ...
       'Textile, Leather, Clothing and Footwear Manufacturing',
       'Transport Equipment Manufacturing', 'Transport Support Services',
       'Transport, Postal and Warehousing',
       'Travel Agency and Tour Arrangement Services',
       'Veterinary and Other Professional Services',
       'Warehousing and Storage Services',
       'Water, Sewerage, Drainage and Waste Services', 'Wholesale Trade',
       'Wood Product Manufacturing'],
      dtype='object', length=119)

#### Filtriranje po kategoriji

In [20]:
nzl[nzl['Industry_name_NZSIOC'] == 'Manufacturing']

,Year,Industry_name_NZSIOC,Units,Variable_name,Variable_category,Value
458,2023,Manufacturing,Dollars (millions),Total income,Financial performance,131578
459,2023,Manufacturing,Dollars (millions),Sales of goods not further processed,Financial performance,7804
460,2023,Manufacturing,Dollars (millions),Sales of other goods and services,Financial performance,122081
461,2023,Manufacturing,Dollars (millions),"Interest, dividends and donations",Financial performance,562
462,2023,Manufacturing,Dollars (millions),"Government funding, grants and subsidies",Financial performance,188
...,...,...,...,...,...,...
46839,2013,Manufacturing,Percentage,Quick ratio,Financial ratios,76
46840,2013,Manufacturing,Percentage,Margin on sales of goods for resale,Financial ratios,27
46841,2013,Manufacturing,Percentage,Return on equity,Financial ratios,10
46842,2013,Manufacturing,Percentage,Return on total assets,Financial ratios,4


#### Promena imena kategorije i izvlačenje novog naziva kategorije

In [30]:
nzl['Industry_name_NZSIOC'] = nzl['Industry_name_NZSIOC'].cat.rename_categories({'Pharmaceutical and Other Store Based Retailing': 'Pharmaceutical Retailing'})
nzl['Industry_name_NZSIOC'].loc[nzl['Industry_name_NZSIOC'].str.startswith('Ph')].value_counts().head(5)

Industry_name_NZSIOC
Pharmaceutical Retailing                                     396
Pharmaceutical, Cleaning and Other Chemical Manufacturing    396
Accommodation                                                  0
Adult, Community and Other Education                           0
Advertising, Market Research and Management Services           0
Name: count, dtype: int64

## Rad sa duplikatima
#### Metoda duplicated() preslikava svaku vrednost datafrejma, (skupa) kolona, ili pandas serije u True ako je duplikat, u suprotnom False. Funkcija podrazumeva da prvo pojavljivanje odr. vrednosti nije duplikat, a sva naredna pojavljivanja tumači kao duplikate.

In [36]:
nzl.duplicated().value_counts()

False    43331
True      7654
Name: count, dtype: int64

#### Kao što je prethodno navedeno, metoda duplicated može da se primeni i nad pojedinačnom kolonom, ili više njih.

In [42]:
nzl['Value'].duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
50980     True
50981     True
50982     True
50983     True
50984     True
Name: Value, Length: 50985, dtype: bool

In [40]:
nzl['Value'].duplicated().value_counts()

Value
True     37312
False    13673
Name: count, dtype: int64

#### U okviru parametra subset možemo navesti kolone (ili samo jednu), na osnovu kojih se posmatraju uređene n-torke po kojima ćemo raditi proveru duplikata.

In [45]:
nzl.duplicated(subset=['Industry_name_NZSIOC','Value']).value_counts()

False    32157
True     18828
Name: count, dtype: int64

#### Duplikate je moguće ukloniti korišćenjem metode drop_duplicates. Ako želimo da promene nad datafrejmom ostanu trajne, postavljamo inplace parametar na True.

In [48]:
print(f'Pre: {nzl.shape}')
print(f'Posle:{nzl.drop_duplicates().shape}')

Pre: (50985, 6)
Posle:(43331, 6)


#### Vidimo da je oko 7000 redova duplikata izbačeno iz datafrejma. Budući da nismo postavili inplace na True, datafrejm idalje ima redpve duplikate u sebi.

In [49]:
nzl.shape

(50985, 6)

#### Ako želimo da uklonimo sve redove koji imaju duplikat kao vrednost odr. kolone, to radimo na sledeći način:

In [50]:
nzl.Value.drop_duplicates()

0         930995
1         821630
2          84354
3          25010
4         832964
          ...   
50972      5,434
50974     10,670
50975     10,212
50976      2,220
50977    523,700
Name: Value, Length: 13673, dtype: object

In [59]:
nzl['Value'].drop_duplicates().duplicated().value_counts()

Value
False    13673
Name: count, dtype: int64

#### Prilikom uklanjanja duplikata podrazumevano je da se čuva prvo pojavljivanje odr. jedinstvene vrednosti. Ukoliko želimo da čuvamo poslednje pojavljivanje, to navodimo u parametru keep (postavljamo ga na last).

In [69]:
nzl.drop_duplicates(['Industry_name_NZSIOC','Value'],keep='last', inplace=True)
nzl[['Industry_name_NZSIOC','Value']].value_counts()

Industry_name_NZSIOC        Value
Wood Product Manufacturing  C        1
Accommodation               0        1
                            1        1
                            1,081    1
                            1,084    1
                                    ..
                            1,486    1
                            1,444    1
                            1,377    1
                            1,373    1
                            1,322    1
Name: count, Length: 32157, dtype: int64

#### Moguće je da indeksi redova imaju duplikate. Pokazaćemo kako se utvrđuje da li je ovaj problem prisutan, kao i kako bi se otklonio. Indeksi našeg dataseta su celobrojne vrednosti, što se vidi iz priloženog.

In [60]:
nzl.index

Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,
       ...
       50975, 50976, 50977, 50978, 50979, 50980, 50981, 50982, 50983, 50984],
      dtype='int64', length=32157)

#### Za identifikaciju duplikata koristićemo već pomenutu metodu duplicated(), i uvešćemo jednu novu.

In [65]:
nzl.index.duplicated()

array([False, False, False, ..., False, False, False])

In [66]:
nzl.index.has_duplicates

False

#### Primećujemo da naš datafrejm ima jedinstvene indekse. Međutim, ukoliko to ne bi bio slučaj, preporuka bi bila da se indeksi resetuju, i postave novi. Postavili smo vrednost True kod parametra drop, kako ne bi čuvao stare indekse. Indeksi su nam se promenili jer sad naš datafrejm ima manje redova (prethodno smo u jednoj naredbi koristili inplace=True pa je deo redova sa duplikatima obrisan).

In [70]:
nzl.reset_index(drop=True)

,Year,Industry_name_NZSIOC,Units,Variable_name,Variable_category,Value
0,2023,All industries,Dollars (millions),Total income,Financial performance,930995
1,2023,All industries,Dollars (millions),"Sales, government funding, grants and subsidies",Financial performance,821630
2,2023,All industries,Dollars (millions),"Interest, dividends and donations",Financial performance,84354
3,2023,All industries,Dollars (millions),Non-operating income,Financial performance,25010
4,2023,All industries,Dollars (millions),Total expenditure,Financial performance,832964
...,...,...,...,...,...,...
32152,2013,Food product manufacturing,Percentage,Quick ratio,Financial ratios,52
32153,2013,Food product manufacturing,Percentage,Margin on sales of goods for resale,Financial ratios,40
32154,2013,Food product manufacturing,Percentage,Return on equity,Financial ratios,12
32155,2013,Food product manufacturing,Percentage,Return on total assets,Financial ratios,5


## Nullable tipovi u pandasu
#### Tipovi koji nisu nullable: int, bool. Svi ostali tipovi su nullable: object, int64, float64, boolean, string, datetime itd.

In [75]:
try:
    df = pd.DataFrame({
        'Int': pd.Series([1, 2, None], dtype='int'),
        'Float': [1.1, 2.2, None],
        'String': pd.Series(['a', None, 'b'], dtype='object'),
        'Object': ['a', None, 'b']
    })
except TypeError as er:
    print(er)

int() argument must be a string, a bytes-like object or a real number, not 'NoneType'


#### Desila se greška u izvršavanju koda jer je tipu int (koji nije nullable) prosleđena None vrednost. Kod bool tipa ne desi se greška, već se None interpretiraju kao False, jer bool ne podržava None tip.

In [83]:
try:
    df = pd.DataFrame({'Bool': [True, None, False]}, dtype='bool')
except TypeError as e:
    print(f"Greška: {e}")
df.Bool

0     True
1    False
2    False
Name: Bool, dtype: bool